<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time
import qcutils

servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))
sr_url=qcutils.read_config_value("kafka.schema_registry.url")

topic = ''

qcutils.create_kafka_topic(topic)

# write a kafka avro producer

Define avro schemas for keys and value to describe cars

### Key
The key has a single field to contain `Plate`

### Value
A car is described by `Manifacturer`, `Model`, `Year`

1) define the key schema

In [ ]:
key_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CarKey",
  "fields": [
    {
      "name": "plate",
      "type": "string"
    }
  ]
}
"""

2) define the value schema

In [ ]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CarValue",
  "fields": [
    {
      "name": "manifacturer",
      "type": "string"
    },
    {
      "name": "model",
      "type": "string"
    },
    {
      "name": "year",
      "type": "int"
    }
  ]
}
"""

3) create a topic `<surname>-avro-cars`, send in messages and read it put with a consumer (can you re-use the previous one?)

In [ ]:
key_schema = avro.loads(key_schema_str)
value_schema = avro.loads(value_schema_str)

In [ ]:
producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

key = {"plate": "ab123cd"}
value = {"manifacturer": "audi", "model": "A1", "year":2018}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)

print(value)

key = {"plate": "ab321cd"}
value = {"manifacturer": "skoda", "model": "karoq", "year":2018}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)

print(value)

ap.flush(1)

## Consume the data

In another notebook, Write a AVRO Consumer that consume the car messages and access directly to the production year...

## Schema Evolution and Value restriction

The different manifacturers added the `Color` to the car description and restricted the number of available colors to 3 (Black, White and Red).

**HINT**: check [AVRO modelling language](https://avro.apache.org/docs/current/idl.html)

Change the schema and write the original topic.

In [ ]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CarEvolvedValue",
  "fields": [
    {
      "name": "manifacturer",
      "type": "string"
    },
    {
      "name": "model",
      "type": "string"
    },
    {
      "name": "year",
      "type": "int"
    },
    {
      "name": "color",
      "type": {
        "type": "enum",
        "name": "possibleColors",
        "symbols": ["no_color","black", "white", "red"]
      },
      "default": "no_color",
      "doc": "The color of the car"
    }
  ]
}
"""

value_schema = avro.loads(value_schema_str)
key_schema = avro.loads(key_schema_str)

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
newap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

key = {"plate": "ab567cd"}
value = {"manifacturer": "fiat", "model": "panda", "year":2020, "color":"black"}
newap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)
key = {"plate": "ab568cd"}
value = {"manifacturer": "fiat", "model": "panda", "year":2020}
newap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.